In [ ]:
!git clone https://github.com/muxspace/facial_expressions

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214 (from 1)
Receiving objects: 100% (14214/14214), 239.65 MiB | 26.71 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [ ]:
import csv
data={}
with open("/content/legend.csv") as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[2].lower()
        if key in data:
            data[key].append(row[1])
        else:
            data[key]=[row[1]]

In [ ]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training/')
os.mkdir('master_data/testing/')


In [ ]:
for emotion in emotion_list:
    os.mkdir(os.path.join('master_data/training/',emotion))
    os.mkdir(os.path.join('master_data/testing/',emotion))

In [ ]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
    train_size = int(split_size*len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    for image in train_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/training', emotion, image)
        copyfile(source, dest)
    for image in train_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/testing', emotion, image)
        copyfile(source, dest)



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(8, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 98, 98, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 49, 49, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 47, 47, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       6,554,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │           8,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,586,408 (25.13 MB)

 Trainable params: 6,586,408 (25.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'
#name of the image converted to an array and one hot encoding of the expression this work done by the image data generator function
train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 10941 images belonging to 8 classes.


In [ ]:
# it is used when epochs 5 and after 5 epochs model don't get update so the iteration gon waste , earlystopping function stop iterating
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01, mode='max')

In [ ]:
model.fit(train_generator,
                    epochs = 10, verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


86/86 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.4517 - loss: 2.8255 - val_accuracy: 0.4605 - val_loss: 1.0794
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


86/86 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.5641 - loss: 1.0050 - val_accuracy: 0.5793 - val_loss: 0.9536
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 209s 2s/step - accuracy: 0.6805 - loss: 0.8598 - val_accuracy: 0.7396 - val_loss: 0.7428
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.7384 - loss: 0.7403 - val_accuracy: 0.7626 - val_loss: 0.7292
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.7513 - loss: 0.7239 - val_accuracy: 0.7622 - val_loss: 0.6878
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.7539 - loss: 0.7058 - val_accuracy: 0.7782 - val_loss: 0.6468
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.7791 - loss: 0.6452 - val_accuracy: 0.7794 - val_loss: 0.6282
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.7764 - loss: 0.6491 - val_accuracy: 0.6837 - val_loss: 0.8434
Epoch 9/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.7386 - loss: 0.7638 - val_accuracy: 0.7411 - val_loss: 0.